In [1]:
!pip install /kaggle/input/librosa/librosa-0.9.1-py3-none-any.whl

Processing /kaggle/input/librosa/librosa-0.9.1-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.6 MB/s eta 0:00:00
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.0.post2
    Uninstalling librosa-0.10.0.post2:
      Successfully uninstalled librosa-0.10.0.post2


In [2]:
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import numpy as np
import librosa
import os

In [3]:
INPUT_PATH = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/'

In [4]:
os.makedirs(f'./defog_np')
os.makedirs(f'./tdcsfog_np')

In [5]:
events = pd.read_csv(f'{INPUT_PATH}/events.csv')
events = events[~events.Type.isnull()]

In [6]:
defog = pd.read_csv(f'{INPUT_PATH}/defog_metadata.csv')
defog = defog[defog.Id.isin(events.Id)]

In [7]:
g0=9.80665

In [8]:
sigs = []

In [9]:
for i,r in defog.iterrows():
    data = pd.read_csv(f'{INPUT_PATH}/train/defog/{r.Id}.csv')
    sig = data[[ 'AccV', 'AccML', 'AccAP']].values
    target = data[['StartHesitation', 'Turn', 'Walking']].values
    
    sig = sig*g0
    np.save(f'./defog_np/{r.Id}_sig.npy',sig)
    np.save(f'./defog_np/{r.Id}_tgt.npy',target)
    sigs.append(sig)

In [10]:
tdcsfog = pd.read_csv(f'{INPUT_PATH}/tdcsfog_metadata.csv')
tdcsfog = tdcsfog[tdcsfog.Id.isin(events.Id)]

In [11]:
for i,r in tdcsfog.iterrows():
    data = pd.read_csv(f'{INPUT_PATH}/train/tdcsfog/{r.Id}.csv')
    sig = data[[ 'AccV', 'AccML', 'AccAP']].values
    target = data[['StartHesitation', 'Turn', 'Walking']].astype(np.float32).values
    
    sig = sig
    sig_resample = []
    
    for i in range(3):
        sig_resample.append(librosa.resample(sig[:,i],orig_sr=128,target_sr=100))
    sig = np.stack(sig_resample,axis=1)
    
        
    target_resample = []
    for i in range(3):
        target_resample.append(librosa.resample(target[:,i],orig_sr=128,target_sr=100))
        
    target = np.stack(target_resample,axis=1)  
    np.save(f'.//tdcsfog_np/{r.Id}_sig.npy',sig)
    np.save(f'.//tdcsfog_np/{r.Id}_tgt.npy',target)
    #print('sig',r.Id,sig)
    sigs.append(sig)
    #print('target',target)
    

In [12]:
np.concatenate(sigs).max(),np.concatenate(sigs).min()

(44.365653391177005, -59.082131744182675)